In [19]:

from datetime import date

from numpy.lib.stride_tricks import as_strided
import process
from numpy.core.fromnumeric import take
import pandas as pd 
import config
import numpy as np 

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
import statistics as stats
from OrderManager import OrderManager
import time
class Strategy():
    
    def __init__(self, orderManager,  take_profit = 10, stop_loss = -4, trade_size = 1):
        self.pnls = []
        self.take_profit = take_profit
        self.stop_loss = stop_loss
        self.open_pnl = 0
        self.close_pnl = 0
        self.position = 0
        self.real_time_price = 0
        self.real_time_volume = 0
        self.trade_size = trade_size
        self.price =  0
        self.total_buy  = 0
        self.total_sell = 0
        self.buy_quantity = 0 
        self.sell_quantity = 0 
        self.orderManager = orderManager
    def getCurrentPnl(self):
        return self.close_pnl + self.open_pnl

    def calculateOpenPnl(self):
        pass

    def signal(self):
        pass
    
    def BuyOder(self, price, quantity): # I plan that the orderManager will be different when we used differennt account in the sercurtites companyies
      
        self.orderManager.send_buy_order(price, quantity)

    def SellOrder(self, price, quantity):
        self.orderManager.send_sell_order(price, quantity)
    
    def StopLossOrder(self, orderManager): # this code is a speical type one I will code it in the last place
        alphaSignal = self.signal()
        orderManager.send_stop_loss_order('MTL', alphaSignal['price'], alphaSignal['quantity'], )

    def enterLongPosition(self):
        pass
    def enterShortPosition(self):
        pass

    def StopLoss(self ,price, quantity):
        if self.open_pnl  <= self.stop_loss:
            if self.position == 1:
                self.SellOrder(price, quantity)
            elif self.position == -1:
                self.BuyOder( price, quantity)

    def TakeProfit(self,price, quantity):
        if self.open_pnl  >= self.take_profit:
            if self.position == 1:
                self.SellOrder( price, quantity)
            elif self.position == -1:
                self.BuyOder( price, quantity)
# we implement the base strategy with some general characters:

# it must stop-loss and take-profit at some specific points # talk about the close position
# the entry point is based on the entry signal from a specific strattegy
# the exit point is based on the signal of overbought or oversold.
# Any technical analysis requires the price and the volume
# we buy and sell , not depend on the order book to find the best price because the 
class MA20(Strategy):
    def __init__(self, orderManager,dateList, time_interval, periods):
        super().__init__(orderManager)
        self.last_position = 0 # this is a mean reversion strategy then we want to know the last_position
        self.MA = self.get_MAs(dateList, time_interval, periods)
    def get_MAs(self, dateList, time_interval, periods):
        prices = process.process_past_data(dateList, time_interval=time_interval)['Price']
        MA_values = []
        MA_history = []

        for i, price in enumerate(prices):
            MA_values.append(price)
            if len(MA_values) > periods:
                del(MA_values[0])
            MA_value = stats.mean(MA_values)
            MA_history.append(MA_value)
        del(MA_values[0])

        return {'History': MA_history, 'LastMAPrice': MA_values}
    
    def update_MA(self, new_value):
        self.MA['LastMAPrice'].append(new_value)
        self.MA['History'].append(stats.mean(self.MA['LastMAPrice']))
        del(self.MA['LastMAPrice'][0])

    def enterLongPosition(self):
        return self.real_time_price > self.MA['LastMAPrice'][-1] # enter long position
    def enterShortPosition(self):
        return self.real_time_price < self.MA['LastMAPrice'][-1] # enter short position
    def signal(self):
        if self.position == 0:
            if self.enterLongPosition() == 1: # return a dict for sell and buy orders
                self.BuyOder(self.real_time_price + 0.5, 1)
                self.position = 1
            elif self.enterShortPosition() == 1:
                self.position = -1
                self.SellOrder( self.real_time_price  - 0.5, 1) # we set the real_time_price - 0.5 to make sure that the order will match
        elif self.position == 1: # we check the stoploss first before check the take_profit
            self.StopLoss('MTL', 1) # prioritize stoploss with the MTL order
            self.TakeProfit(self.real_time_price + 0.4, 1)
        elif self.position == -1:
            self.StopLoss('MTL', 1) # prioritize stoploss with the MTL order
            self.TakeProfit(self.real_time_price + 0.4, 1)
        
    def getMatchedPrice(self):
        
        if self.orderManager.checkOrderStatus() == 'match':
            currentPrice = self.orderManager.getMatchedPrice().text.replace(',','')
            self.price = float(currentPrice)
            
    def calculateOpenPnl(self):
        if self.position == 0:
            self.open_pnl = 0
            return 0;
        elif self.position == 1:
            self.open_pnl = self.real_time_price - self.price
            return self.open_pnl
        else:
            self.open_pnl = self.real_time_price - self.price


In [2]:
PATH=r"C:\Users\ThanhDat\Desktop\worldquant 2021\chromedriver.exe"
def openProtrial():
    driver = webdriver.Chrome(PATH)
    driver.get("https://protrade-trial.vndirect.com.vn/trang-chu") 
    return driver

driver = openProtrial()
time.sleep(5)
checkbox=  driver.find_element_by_xpath("/html/body/div/div/div[7]/section/div[2]/div/p/input")
checkbox.click()

confirmButton = driver.find_element_by_xpath("/html/body/div/div/div[7]/section/div[2]/div/button")
confirmButton.click()

time.sleep(3)
my_username = "toiptune@viettel.com.vn"
my_password = "luongducphat1977"
username = driver.find_element_by_name('username')
password = driver.find_element_by_name('password')

username.send_keys(my_username)
password.send_keys(my_password)


driver.find_element_by_xpath('/html/body/div/div/div[2]/div/section/div[2]/div/form/button').click()

print("Succesfully login")

Succesfully login


In [7]:
orderManager = OrderManager(driver)

In [8]:
orderManager.send_sell_order('ATO',1)

In [11]:
orderManager.refreshPage()

In [12]:
orderManager.checkOrderStatus()

'error'

In [13]:
process.process_past_data(['2021-07-20'],1)

2021-07-23 08:26:03,565 : INFO : NumExpr defaulting to 8 threads.


,1minute,Price,Volume,TimeInterval
0,540,1373.487623,3757.0,2021-07-20 9:0
1,541,1374.266707,1637.0,2021-07-20 9:1
2,542,1375.012607,1396.0,2021-07-20 9:2
3,543,1374.322040,1284.0,2021-07-20 9:3
4,544,1375.430796,1419.0,2021-07-20 9:4
...,...,...,...,...
238,867,1407.972480,1250.0,2021-07-20 14:27
239,868,1408.517477,1625.0,2021-07-20 14:28
240,869,1408.543227,2045.0,2021-07-20 14:29
241,870,1408.500000,228.0,2021-07-20 14:30


In [20]:
maStrategy = MA20(orderManager,['2021-07-20'], 1, 20)

In [16]:
gay  = Strategy(orderManager)

In [21]:
maStrategy.MA

{'History': [1373.48762310354,
  1373.8771652475548,
  1374.2556459816556,
  1374.272244610852,
  1374.5039549557712,
  1374.705427365752,
  1374.9044799107478,
  1375.0159173278707,
  1375.1827024044387,
  1375.3780904741154,
  1375.6317882258388,
  1375.8093293406264,
  1376.0354148335352,
  1376.188418082387,
  1376.2814139996026,
  1376.4949701655703,
  1376.6922594052637,
  1376.8161617500748,
  1376.897740719132,
  1376.9283775702547,
  1377.0633492879738,
  1377.0779019426063,
  1377.0305894412072,
  1377.0715145098811,
  1377.0903046250814,
  1377.1181020212805,
  1377.1829703803255,
  1377.2821828889241,
  1377.3618744079315,
  1377.3844881626317,
  1377.3880336341788,
  1377.3856481972305,
  1377.2882261598083,
  1377.1877685100817,
  1377.1003407810451,
  1376.9755196861058,
  1376.8719497829625,
  1376.81372836587,
  1376.7795387987953,
  1376.7978387082646,
  1376.9035685939893,
  1377.118609141207,
  1377.3387315686205,
  1377.5286674906718,
  1377.7079482457218,
  1377.9